In [1]:
# Notebook to compute, partition, and invert quasigeostrophic potential vorticity
# For MEA 717, spring 2022, Gary Lackmann
# Kudos to Michael Gray for contributing a much faster method of computing Laplacians
#
# This code computes the QG PV vorticity, partitions it, and inverts specified pieces
#
# Some of the following imports are not used right now, but will retain for future flexibility
import os
import subprocess
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.cbook as cbook
from matplotlib.colors import Normalize
import matplotlib.colors as mcolors

import pandas as pd
import xarray as xr
import numpy as np
import math
from numpy import *
from pylab import *
import pyproj

from siphon.catalog import TDSCatalog
from siphon.http_util import session_manager
from datetime import datetime, timedelta
from xarray.backends import NetCDF4DataStore
from netCDF4 import Dataset
import metpy as metpy
import metpy.calc as mpcalc
from metpy.plots import ctables
from metpy.units import units
from metpy.plots import add_metpy_logo, add_timestamp
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter

import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
from cartopy import config
import wrf
from wrf import (to_np, interplevel, geo_bounds, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)
# Download and add the states and coastlines
states = NaturalEarthFeature(category="cultural", scale="50m",
                          facecolor="none", name="admin_1_states_provinces_shp")
import glob

In [2]:
########### Define the colors in the desired sequence for IVT: ###########

ivt_colors = ['#FFFF00', '#FFEE00','#FFDC00', '#FFB700',
          '#FFA300', '#FF9000', '#FF7D00', '#FF6800',
          '#FF5200', '#C70039','#900C3F', (.88,.24,.69)]
cmap='YlGnBu'
############## Create a colormap using ListedColormap #################

ivt_cmap = mcolors.ListedColormap(ivt_colors)

ivt_levels = np.arange(250,1150,50)

# Create levels based on the number of colors
start, end = 250, 1150
num_intervals = len(ivt_colors) - 1

# linspace creates an array of evenly spaced numbers over a specified range
#ivt_levels = np.linspace(start, end, 12)
pmsl_levels = np.arange(960, 1060, 4)
# dark brown for state/coastlines
dark_brown = (0.4, 0.2, 0)


In [3]:
datafiles = (glob.glob("/scratch/sawyer/wwrf/2017-01-09/ishmael/wrfout_d01_2017-01-08_12:00:00"))
datafiles.sort()
numfiles=len(datafiles)
ncfile = Dataset(datafiles[0])
p = getvar(ncfile, "pressure")
# Get the lat/lon coordinates
wrf_lats, wrf_lons = latlon_coords(p)

## WWRF longitude fix ######
new_lons =np.where(wrf_lons > 0, wrf_lons - 360, wrf_lons)

# Sigma for gaussian filter setting for SLP
sigma = 3


In [4]:
# Initialize an empty list to store the filtered ivt arrays
ivt_list = []
slp_list = []
# Get a list of all .npy files in the directory
file_paths = glob.glob('/scratch/sawyer/wwrf/2017-01-09/ensemble_data/total_ivt/ivt_*_20170109_00.npy')
slp_paths = glob.glob('/scratch/sawyer/wwrf/2017-01-09/ensemble_data/slp/slp_*_20170109_00.npy')
# Sort the file paths alphabetically
sorted_file_paths = sorted(file_paths)
# Loop through each file and load, filter, and append the data
for file in sorted_file_paths:
    print(file)
    ivt_arrays = np.load(file, allow_pickle=True)
    ivt_list.append(ivt_arrays)
    #print(ivt_arrays.shape)
sorted_slp_paths = sorted(slp_paths)
# Loop through each file and load, filter, and append the data
for file in sorted_slp_paths:
    print(file)
    slp_arrays = np.load(file, allow_pickle=True)
    slp_array = gaussian_filter(slp_arrays, sigma)
    slp_list.append(slp_array)
    #print(ivt_arrays.shape)

/scratch/sawyer/wwrf/2017-01-09/ensemble_data/total_ivt/ivt_ishamel_20170109_00.npy
/scratch/sawyer/wwrf/2017-01-09/ensemble_data/total_ivt/ivt_ntu_20170109_00.npy
/scratch/sawyer/wwrf/2017-01-09/ensemble_data/total_ivt/ivt_p3_20170109_00.npy
/scratch/sawyer/wwrf/2017-01-09/ensemble_data/total_ivt/ivt_p3_2nd_20170109_00.npy
/scratch/sawyer/wwrf/2017-01-09/ensemble_data/total_ivt/ivt_p3_2xcloud_20170109_00.npy
/scratch/sawyer/wwrf/2017-01-09/ensemble_data/total_ivt/ivt_p3mom_20170109_00.npy
/scratch/sawyer/wwrf/2017-01-09/ensemble_data/total_ivt/ivt_thompson_20170109_00.npy
/scratch/sawyer/wwrf/2017-01-09/ensemble_data/total_ivt/ivt_thopmson_aa_20170109_00.npy
/scratch/sawyer/wwrf/2017-01-09/ensemble_data/total_ivt/ivt_wdm6_20170109_00.npy
/scratch/sawyer/wwrf/2017-01-09/ensemble_data/total_ivt/ivt_wsm6_20170109_00.npy
/scratch/sawyer/wwrf/2017-01-09/ensemble_data/slp/slp_ishamel_20170109_00.npy
/scratch/sawyer/wwrf/2017-01-09/ensemble_data/slp/slp_ntu_20170109_00.npy
/scratch/sawyer/ww

In [5]:
print(len(ivt_list))
# Make a list of title for plotting
titles = ['Ishmael','NTU','P3','P3 2nd', 'P3 2X', 'P3 3-mom',
          'Thompson', 'Thompson AA', 'WDM6', 'WSM6',
           'WWRF Reanalysis']


10


In [6]:
#datafiles = (glob.glob("/scratch/sawyer/wwrf/2017-01-09/ishmael/wrfout_d01_2017-01-08_12:00:00"))
datafiles = glob.glob("/scratch/sawyer/wwrf/2017-01-09/wwrf_6km_reanalysis/clc/WRF-CA-CLC_subset_d01_2017-01-08_00_00_00.nc")
datafiles.sort()
numfiles=len(datafiles)
print(numfiles)
print(datafiles[0])
# Assuming you only want to inspect the first file

if datafiles:
    nc_file = datafiles[0]
    with Dataset(nc_file, 'r') as nc:
        print("Variables in the .nc file:")
        for variable in nc.variables:
            print(variable)
else:
    print("No files found matching the pattern.")


1
/scratch/sawyer/wwrf/2017-01-09/wwrf_6km_reanalysis/clc/WRF-CA-CLC_subset_d01_2017-01-08_00_00_00.nc
Variables in the .nc file:
XTIME
lon
lat
XLAND
HGT
PSFC
T2
Q2
U10
V10
SWDOWN
SOLZEN
ALBEDO_BCK
ALBEDO_SFC
ALBEDO_ALL
TPW
SLP
SST
UHub
VHub
GUST_UST
GUST_TKE
THETA_1000
U1000
V1000
Z1000
Q1000
THETA_925
U925
V925
Z925
Q925
THETA_850
U850
V850
Z850
Q850
THETA_700
U700
V700
Z700
Q700
THETA_500
U500
V500
Z500
Q500
THETA_300
U300
V300
Z300
Q300
QCLOUD_925
QCLOUD_700
QCLOUD_500
Z0C
LCL
PBLH
IBH
ITH
IBP
ISTR
CWP
CWP700
LWCLD_TKE
LHF
SHF
SMOIS
AET
CAPE
CIN
CTT
HainesLow
HainesMid
HainesHigh


In [9]:
p = ncfile['HGT']
print(p.shape)

(1, 942, 1957)


In [8]:
# Set up loop to run same plots for multiple different times (assume here that we have 1 time per wrfout file)
#for j in range(0,numfiles):

#for j in range(0,numfiles):
ncfile = Dataset(datafiles[0])# ncfile = Dataset(datafiles[j])
Time=wrf.extract_times(ncfile, timeidx=0, method='cat', squeeze=True, cache=None, meta=False, do_xtime=False)
timestr=(str(Time))
# Set up one time string for plot titles, another for file names
titletime=(timestr[0:10]+' '+timestr[11:16])
filetime=(timestr[0:10]+'_'+timestr[11:13])
print('WRF valid time: ',filetime)


slp = ncfile['slp'][0]

slp_smoothed = gaussian_filter(slp, sigma)

ivt = ncfile['IVT'][0]
lat =ncfile['lat'][:]
print(lat.shape)
lon = ncfile['lon'][:]
print(lon.shape)
proj = ncfile['Lambert_Conformal']
print(proj)
# Define the Lambert Conformal Conic projection using the given parameters
lcc_projection = ccrs.LambertConformal(
    central_longitude=-125.0,
    central_latitude=40.99998,
    standard_parallels=(50, 32))
# Download and add the states and coastlines
states = NaturalEarthFeature(category="cultural", scale="50m",facecolor="none", name="admin_1_states_provinces_shp")
# Get the latitude and longitude points


## WWRF longitude fix ######
lons =np.where(lon > 0, lon- 360, lon)

pmsl_levels = np.arange(960, 1060, 4)
fig, ax = plt.subplots(figsize=(14,10), subplot_kw={'projection': lcc_projection})
ax.set_extent([-150, -110, 15, 60])
ax.add_feature(cfeature.STATES.with_scale('50m'), linewidth=0.25, edgecolor=dark_brown)
ax.coastlines('50m', linewidth=0.6, color=dark_brown)
ax.add_feature(cfeature.BORDERS.with_scale('50m'), linestyle=':', linewidth=0.5)


cs = plt.contour(to_np(lons), to_np(lat), to_np(slp_smoothed),
                         colors = 'black',levels=pmsl_levels, linewidths=.7, alpha=1,transform=crs.PlateCarree())
plt.clabel(cs, fmt='%1.0f', inline=True,levels=pmsl_levels)
cs1 = plt.contourf(to_np(lons), to_np(lat), to_np(ivt), levels=ivt_levels, cmap=cmap,transform=ccrs.PlateCarree())
cbar = plt.colorbar(cs1, ax=ax, orientation="horizontal", pad=.03, shrink=.8, aspect=50)
cbar.set_label("IVT (kg m$^{-1}$ s$^{-1}$)", fontsize = 12)
#cbar.set_label("Precipitation (mm)", fontsize = 16)
#rain_contours = plt.contourf(to_np(lons), to_np(lats), to_np(raintot), levels=rain_levels,
#                         cmap=get_cmap("rainbow"),norm=Normalize(0,25), vmin=0, vmax=25, alpha=.5,transform=crs.PlateCarree())
#plt.colorbar(rain_contours, ax=ax, orientation="horizontal", pad=.03, shrink=.8, aspect=50)

plt.title("SLP and IVT")
plt.show()
plt.close(fig)

    

WRF valid time:  None_


IndexError: slp not found in /

In [ ]:
ivt_list.append(ivt)
slp_list.append(slp_smoothed)

In [ ]:
print(len(ivt_list))
print(len(slp_list))

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(24, 16),
                        subplot_kw={'projection': ccrs.LambertConformal()})
axs = axs.ravel()  # Flatten axs

# Loop through the first N-1 elements
for i in range(len(ivt_list) - 1):
    ax = axs[i]
    ax.set_extent([-140, -113, 25, 50])
    ax.add_feature(cfeature.COASTLINE, linewidth=1.0, edgecolor='black')
    ax.add_feature(cfeature.STATES, linewidth=1.0, edgecolor='black')
    ax.set_title(titles[i], fontweight='bold')
    cp = ax.contourf(to_np(new_lons), to_np(wrf_lats), to_np(ivt_list[i]), cmap=cmap, 
                     levels=ivt_levels, transform=ccrs.PlateCarree())
    cp1 = ax.contour(to_np(new_lons), to_np(wrf_lats), to_np(slp_list[i]),
                         colors = 'black',levels=pmsl_levels, linewidths=.7, alpha=1,transform=crs.PlateCarree())
    ax.clabel(cp1, fmt='%1.0f', inline=True,levels=pmsl_levels)


# Handle the last plot separately
ax = axs[len(ivt_list) - 1]
ax.set_extent([-140, -113, 25, 50])
ax.add_feature(cfeature.COASTLINE, linewidth=1.0, edgecolor='black')
ax.add_feature(cfeature.STATES, linewidth=1.0, edgecolor='black')
ax.set_title(titles[-1], fontweight='bold')
cp = ax.contourf(to_np(lons), to_np(lat), to_np(ivt_list[-1]), cmap=cmap, 
                 levels=ivt_levels, transform=ccrs.PlateCarree())
cp2 = ax.contour(to_np(lons), to_np(lat), to_np(slp_list[-1]),
                         colors = 'black',levels=pmsl_levels, linewidths=.7, alpha=1,transform=crs.PlateCarree())
ax.clabel(cp2, fmt='%1.0f', inline=True,levels=pmsl_levels)

# Turn off any remaining subplots
for i in range(len(ivt_list), len(axs)):
    axs[i].axis('off')
# Add a single colorbar at the bottom of the figure
cbar_ax = fig.add_axes([0.15, 0.08, 0.7, 0.02])  # Adjust these dimensions to fit the colorbar nicely
cbar = fig.colorbar(cp, cax=cbar_ax, orientation='horizontal')
cbar.set_label("IVT (kg m$^{-1}$ s$^{-1}$)", fontsize = 12)
# Adjust layout
fig.tight_layout()
plt.subplots_adjust(bottom=0.15)  # Make space for the colorbar

plt.savefig('ivt_00z_9Jan')
plt.show()

In [ ]:
# Create 11 subplots
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(20, 12),
                        subplot_kw={'projection': ccrs.LambertConformal()}) #ccrs.PlateCarree()
axs = axs.ravel()  # Flatten axs to loop over
# Gridlines' latitudes and longitudes
lats = np.arange(32.5, 43, 2)  # Example: ticks every 2 degrees. Adjust as needed.
lons = np.arange(-125, -114, 2)
for i, ax in enumerate(axs):
    #print(titles[i])
    ax.set_extent([-150,-113,15,60])
    ax.add_feature(cfeature.COASTLINE, linewidth=1.0, edgecolor='black')  # Add coastlines
    ax.add_feature(cfeature.STATES, linewidth=1.0, edgecolor='black')
    ax.coastlines()
    # Use gridlines and customize them to look like ticks
    #gl = ax.gridlines(draw_labels=False, linewidth=1, color='black', alpha=0.5, linestyle='--', xlocs=lons, ylocs=lats)
    #gl.xlabels_bottom = True
    #gl.ylabels_left = True
    #gl.xlines = True
    #gl.ylines = True
    #gl.linewidth = .5
    #gl.color = 'black'

    # Set the title for each subplot
    ax.set_title(titles[i], fontweight='bold')
    if i < len(ivt_list)-1:  # Only use this block for the first N-1 elements
    # Plot filled contour
        cp = ax.contourf(to_np(new_lons),to_np(wrf_lats), to_np(ivt_list[i]), cmap=cmap, 
                         levels=ivt_levels, transform=ccrs.PlateCarree())
    # Plot the 11th element using lons and lat
    elif i == len(ivt_list) - 1:
        cp = ax.contourf(to_np(lons), to_np(lat), to_np(ivt_list[i]), cmap=cmap, 
                         levels=ivt_levels, transform=ccrs.PlateCarree())
    # Turn off any remaining axes if there are more than 11
    else:
        ax.axis('off')
# Turn off the last subplot
#axs[-1].axis('off')
# Add a single colorbar at the bottom of the figure
cbar_ax = fig.add_axes([0.15, 0.08, 0.7, 0.02])  # Adjust these dimensions to fit the colorbar nicely
cbar = fig.colorbar(cp, cax=cbar_ax, orientation='horizontal', label="Precipitation (mm)")
cbar.set_label("Precipitation (mm)", fontproperties=font_bold)
# Adjust layout
fig.tight_layout()
plt.subplots_adjust(bottom=0.15)  # Make space for the colorbar

plt.savefig('ivt_00z_8Jan')
plt.show()